reference.

https://python.langchain.com/docs/expression_language/get_started

In [ ]:
%pip install faiss-cpu

In [11]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

import os
os.environ["OPEN_API_KEY"] = "my key"


Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors.

search result will be fetched by retriever and it'll be used as context

In [12]:
vectorstore = FAISS.from_texts( 
    ["Everyone said one plus one equals three", "Dad said one plus one equals four"], 
    embedding=OpenAIEmbeddings(openai_api_key=os.environ["OPEN_API_KEY"])
)

retriever = vectorstore.as_retriever()

make a template with context

In [13]:

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"], model_name="gpt-4-0125-preview")
output_parser = StrOutputParser()


context include custom test resutls fetched by the retriever. 
question will contain user's question.

In [14]:

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("what is one plus one based on my Dad?")

'Based on your Dad, one plus one equals four.'